In [1]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os

In [2]:
data_dir = '/media/real/Samsung_T5/Trash/real_images/real_images'

batch_size = 128
workers = 0 if os.name == 'nt' else 8

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [4]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [5]:
print('Step 1 ...')
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
print('Step 2 ...')
dataset.samples = [
    (p, p.replace(data_dir, data_dir + '_cropped'))
        for p, _ in dataset.samples
]

print('Step 3 ...')

loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

for i, (x, y) in enumerate(loader):
    # Detect faces in the batch, `mtcnn.detect` returns face bounding boxes and probabilities
    batch_boxes, probs = mtcnn.detect(x)
    
    for img_idx, prob in enumerate(probs):
        # Skip images with no detected faces
        if prob is None:
            print(f"No face detected in image {i * batch_size + img_idx}")
            continue  # Skip if no faces detected
        
        # Save only if faces are detected
        mtcnn(x[img_idx], save_path=y[img_idx])
print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')
    
# Remove mtcnn to reduce GPU memory usage
del mtcnn

Step 1 ...
Step 2 ...
Step 3 ...
Batch 3925 of 3925